In [1]:
from __future__ import annotations
import json
import pandas as pd
import json
import numpy as np
from typing import Iterable
from dataclasses import dataclass, field

# http://etetoolkit.org/download/
# http://etetoolkit.org/docs/latest/tutorial/tutorial_ncbitaxonomy.html#setting-up-a-local-copy-of-the-ncbi-taxonomy-database
# warning: ete tk inserts itself into ~/.local/share/ete/
from ete4 import NCBITaxa
ncbi = NCBITaxa()

from local.utils import regex

In [2]:
_df = pd.read_csv("../../data/scadc_nr.tsv", header=None, sep="\t")
print(_df.shape)
hits = {}
meta = {}
for _, row in _df.iterrows():
    query, subject, annotation, pident, bitscore = row
    hits[query] = hits.get(query, []) + [(subject, pident, bitscore)]

    if subject in meta: continue
    meta[subject] = annotation

for q, d in hits.items():
    hits[q] = sorted(d, key=lambda x: x[2], reverse=True)

len(hits), len(meta)

(98034, 5)


(4127, 93071)

In [3]:
_rows = []
for i, (q, d) in enumerate(hits.items()):
    # print(f"{i}/{len(hits)}", end="\r")
    best = d[0]
    subject, pident, bitscore = best

    ann = meta[subject]
    tax = next(regex(r"\[.+\]", ann))[1:-1]

    _rows.append((q, tax, pident, bitscore))

_df = pd.DataFrame(_rows, columns=["query", "tax", "pident", "bitscore"])

In [4]:
def tax_from_sci_name(sci_name):
    _get = lambda x: next(iter(x.values()))
    tax_data = ncbi.get_name_translator([sci_name])
    if len(tax_data)==0: # retry with genus
        tax_data = ncbi.get_name_translator([sci_name.split(" ")[0]])
    if len(tax_data)==0:
        taxid, match_type = None, None
    else:
        taxid = _get(tax_data)[0]
        match_type = _get(ncbi.get_rank([taxid]))
    return taxid, match_type

In [5]:
print(_df.shape)
_rows = []
for _, row in _df.iterrows():
    q, tax, pident, bitscore = row
    taxid, match_type = tax_from_sci_name(tax)
    if taxid is None: continue
    id_lin = ncbi.get_lineage(taxid)
    assert id_lin is not None
    names = ncbi.get_taxid_translator(id_lin)
    ranks = ncbi.get_rank(id_lin)
    lineage = [(ranks[id], names[id]) for id in id_lin if ranks[id] != "no rank"]

    _rows.append((q, tax, taxid, pident, bitscore, json.dumps(lineage, separators=(',', ':'))))
_df = pd.DataFrame(_rows, columns=['query', 'sci_name', 'taxid', 'pident', 'bitscore', 'lineage_json'])
print(_df.shape)

(4127, 4)
(4096, 6)


In [6]:
tax_from_sci_name("Escherichia coli")

(562, 'species')

In [7]:
from scipy.stats import chi2

In [18]:
freedom = 1
# chi2.cdf(0.95, freedom) - chi2.cdf(0.05, freedom)
# 1 - chi2.cdf(2.70554, freedom)
1 - chi2.cdf(14, freedom)

# chi2.ppf(0.95, freedom)

0.00018281063298186684

In [24]:
from __future__ import annotations
from dataclasses import dataclass, field
from typing import Iterable
import numpy as np
# http://etetoolkit.org/download/
# http://etetoolkit.org/docs/latest/tutorial/tutorial_ncbitaxonomy.html#setting-up-a-local-copy-of-the-ncbi-taxonomy-database
# warning: ete tk inserts itself into ~/.local/share/ete/
from ete4 import NCBITaxa

_ete_ncbi = None
def InitNcbi() -> NCBITaxa:
    global _ete_ncbi
    if _ete_ncbi is None:
        _ete_ncbi = NCBITaxa()
    return _ete_ncbi

class Lineage:
    @classmethod
    def FromTaxID(cls, taxid: int) -> Lineage|None:
        ncbi = InitNcbi()
        id_lin = ncbi.get_lineage(taxid)
        if id_lin is None: return None
        names = ncbi.get_taxid_translator(id_lin)
        ranks = ncbi.get_rank(id_lin)
        return cls((ranks[id], names[id]) for id in id_lin if ranks[id] != "no rank")

    @classmethod
    def FromSciName(cls, sci_name: str) -> Lineage|None:
        ncbi = InitNcbi()
        _get = lambda x: next(iter(x.values()))
        tax_data = ncbi.get_name_translator([sci_name])
        if len(tax_data)==0: # retry with genus
            tax_data = ncbi.get_name_translator([sci_name.split(" ")[0]])
        if len(tax_data)==0:
            return None
        else:
            taxid = _get(tax_data)[0]
        return cls.FromTaxID(taxid)

    def __init__(self, lineage: Iterable[tuple[str, str]]|None = None) -> None:
        """@lineage is iterable of (name, level) tuples"""
        self._i = 0
        self._lineage: list[tuple[str, str]] = [] if lineage is None else list(lineage)

    def Add(self, name: str, level: str):
        self._lineage.append((name, level))

    def __iter__(self):
        self._i = 0
        return self
    
    def __next__(self):
        if self._i >= len(self._lineage): raise StopIteration
        self._i += 1
        return self._lineage[self._i-1]
    
@dataclass
class ResultNode:
    name: str
    level: str
    entropy: float
    cumulative_votes: int
    fraction_votes: float
    p_value: float

class LcaStar:
    @dataclass
    class Node:
        name: str
        level: str
        entropy: float =    field(default_factory=lambda: 0.0)
        count: int =        field(default_factory=lambda: 0)
        sum_entropy: float =field(default_factory=lambda: 0.0)
        sum_count: int =    field(default_factory=lambda: 0)
        parent: LcaStar.Node|None = field(default_factory=lambda: None)

        def __str__(self) -> str:
            return f"<{self.name}, entropy={self.sum_entropy}, count={self.sum_count}>"
        
        def __repr__(self) -> str:
            return self.__str__()
        
    def __init__(self) -> None:
        self.root = LcaStar.Node("root", "root")
        self.nodes: dict[str, LcaStar.Node] = {self.root.name:self.root}
        self.sum = 0
        self.counts = set()

    def NewObservation(self, lineage: Lineage):
        self.sum += 1
        parent = self.root
        for k, level in lineage:
            if k not in self.nodes:
                self.nodes[k] = LcaStar.Node(k, level, parent=parent)
            else:
                assert self.nodes[k].parent == parent, f"invalid lineage: the parent of [{k}] was previously said to be [{self.nodes[k].parent}], now [{parent.name}]"
            parent = self.nodes[k]
        leaf = parent # last node assigned in the loop
        leaf.count += 1

    def BestLineage(self) -> list[ResultNode]:
        # -----------------------------------------------------
        # update entropy

        for node in self.nodes.values():
            node.entropy = 0.0

        for node in self.nodes.values():
            if node.count == 0: continue

            self.counts.add(node.count)
            r_of_x = node.count/self.sum
            c = node.count
            entropy = r_of_x*np.log10(r_of_x)
            while node is not None:
                node.sum_entropy += entropy
                node.sum_count += c
                node = node.parent

        # -----------------------------------------------------
        # prepare for finding best lineage

        all_children: dict[str, list[LcaStar.Node]] = {}
        for node in self.nodes.values():
            if node.parent is None: continue
            if node.parent.name not in all_children:
                all_children[node.parent.name] = []
            all_children[node.parent.name].append(node)
        
        # for calculating p-value
        other_roots: list[LcaStar.Node] = []
        _last_len, _last_largest_other = 0, 0
        others: list[LcaStar.Node] = [] # going down tree, in case some counts are not at leaf
        def _find_second_largest_count():
            nonlocal _last_len, _last_largest_other
            _get_max = lambda: max(max(n.count for n in others), _last_largest_other)
            if _last_len == len(other_roots): return _get_max()
            todo = other_roots.copy()
            while len(todo)>0:
                node = todo.pop()
                if node.count > _last_largest_other:
                    _last_largest_other = node.count
                todo += all_children.get(node.name, [])
            _last_len = len(other_roots)
            return _get_max()

        # p-value is based on supremacy, where null hypothesis is:
        # there actially exists another taxon with a count larger than @node.count
        def _pvalue(node: LcaStar.Node, second_largest_count: int) -> float:
            M = float(second_largest_count) # "M" from supplementary
            X_k = float(node.sum_count) # "X_k" from supplementary
            if X_k <= M:
                t = 0.0 # trivial case
            else:
                first = 0
                if M > 0:
                    first = M * np.log10( (2 * M) / (M + X_k) )
                second = X_k * np.log10( (2 * X_k) / (M + X_k) )
                t = 2.0*(first + second)
            # "...it can also be shown that an approximate likelihood-ratio test at some significance level α
            # can be approximated by a χ-square distribution of degree one"
            # https://doi.org/10.1198/jasa.2009.tm08213
            degrees_of_freedom = 1
            return float(1 - chi2.cdf(t, degrees_of_freedom))

        # -----------------------------------------------------
        # find best lineage

        lineage: list[ResultNode] = []
        node = self.root
        children = all_children[node.name]
        while True:
            entropy_ordered = sorted(children, key=lambda x: x.sum_entropy)
            best = entropy_ordered[0]
            if len(entropy_ordered)>1:
                for other in entropy_ordered[1:]:
                    other_roots.append(other)
            if best.parent is not None: others.append(best.parent) 

            lineage.append(ResultNode(
                name                = best.name,
                level               = best.level,
                entropy             = best.sum_entropy,
                cumulative_votes    = best.sum_count,
                fraction_votes      = best.sum_count/self.sum,
                p_value             = _pvalue(best, _find_second_largest_count()),
            ))
            if best.name not in all_children: break
            children = all_children[best.name]
        return lineage

trees: dict[str, LcaStar] = {}
ranks = {}
for _, row in _df.iterrows():
    query, sci_name, taxid, pident, bitscore, lineage_json = row
    entry = query.split("_")[0]
    lineage = Lineage()
    for rank, name in json.loads(lineage_json):
        ranks[name] = rank
        lineage.Add(name, rank)
    if entry not in trees:
        trees[entry] = LcaStar()
    trees[entry].NewObservation(lineage)

lineages: dict[str, list[ResultNode]] = {}
for i, (k, tree) in enumerate(trees.items()):
    # if i != 3: continue
    lin = tree.BestLineage()
    lineages[k] = lin
    # print(k)
    # print([(n.level, n.name) for n in lin if n.level not in {"clade", }])
    # print([(n.cumulative_votes) for n in lin])
    # print([(n.fraction_votes) for n in lin])
    # print([(round(n.entropy*100)/100) for n in lin])
    # print([(round(n.p_value*10000)/10000) for n in lin])
    # print()

    # break

In [25]:
import json
_rows = []
tax_levels = "superkingdom, phylum, class, order, family, genus, species".split(", ")
for k, lin in lineages.items():
    by_level = {n.level: n for n in lin}
    nodes =     [by_level[l] if l in by_level else None for l in tax_levels]
    tax =       [n.name if n is not None else None for n in nodes]
    pvalues =   [round(n.p_value, 5) if n is not None else None for n in nodes]
    votes =     [n.cumulative_votes if n is not None else None for n in nodes]
    frac_votes =[round(n.fraction_votes, 3) if n is not None else None for n in nodes]
    entropy =   [round(n.entropy, 3) if n is not None else None for n in nodes]

    entry = [k]+tax+pvalues+votes+frac_votes+entropy+[json.dumps([n.__dict__ for n in lin], separators=(",", ":"))]
    _rows.append(entry)

stats = [None, "pvalue", "votes", "frac_votes", "entropy"]
tax_cols = [f"{l}_{s}" if s is not None else l for s in stats for l in tax_levels]
cols = ["fosmid"] + tax_cols + ["raw"]
df = pd.DataFrame(_rows, columns=[c for c in cols])
print(df.shape)
df.to_csv("./cache/scadc_nr_lca_star.csv", index=False)
df.head()

(123, 37)


,fosmid,superkingdom,phylum,class,order,family,genus,species,superkingdom_pvalue,phylum_pvalue,...,genus_frac_votes,species_frac_votes,superkingdom_entropy,phylum_entropy,class_entropy,order_entropy,family_entropy,genus_entropy,species_entropy,raw
0,OP713741.1,Bacteria,Bacteroidota,Bacteroidia,Bacteroidales,None,None,Bacteroidales bacterium 6E,0.00014,0.00014,...,NaN,0.417,-0.295,-0.295,-0.295,-0.158,NaN,NaN,-0.158,"[{""name"":""Bacteria"",""level"":""superkingdom"",""en..."
1,OP713740.1,Bacteria,Bacillota,Clostridia,Eubacteriales,None,None,Eubacteriales bacterium,0.00006,0.00006,...,NaN,0.407,-0.578,-0.578,-0.578,-0.418,NaN,NaN,-0.159,"[{""name"":""Bacteria"",""level"":""superkingdom"",""en..."
2,OP713739.1,Archaea,Euryarchaeota,Methanomicrobia,Methanotrichales,Methanotrichaceae,Methanothrix,Methanothrix sp.,0.00000,0.00000,...,1.000,0.381,-0.447,-0.447,-0.447,-0.447,-0.447,-0.447,-0.160,"[{""name"":""Archaea"",""level"":""superkingdom"",""ent..."
3,OP713738.1,Bacteria,Chloroflexota,Anaerolineae,Anaerolineales,Anaerolineaceae,Anaerolinea,Anaerolinea thermolimosa,0.00002,0.07448,...,0.200,0.167,-0.830,-0.604,-0.456,-0.326,-0.326,-0.179,-0.130,"[{""name"":""Bacteria"",""level"":""superkingdom"",""en..."
4,OP713737.1,Bacteria,Thermodesulfobacteriota,Syntrophia,Syntrophales,Syntrophaceae,Syntrophus,Syntrophus aciditrophicus,0.00014,0.00644,...,0.583,0.333,-1.026,-0.821,-0.616,-0.616,-0.616,-0.501,-0.240,"[{""name"":""Bacteria"",""level"":""superkingdom"",""en..."


In [30]:
n_ar = len(df[df.superkingdom == "Archaea"])
n_ar / len(df)

0.21138211382113822